# Pokušaj poboljšanja bota

In [1]:
import requests
import models
import os
from dotenv import load_dotenv
import util
import bot_my_version
import pandas as pd

In [2]:
data_handler = util.DataHandler()

Dodajem pomoćnu metodu za accuracy, ovdje ću lokalno testirati "bot_my_version" jer mi je jednostavnije, ali naravno možemo ga jednostavno zamijeniti sa originalnim botom i hostati

In [3]:
def get_accuracy(x, y):
    count = 0
    for message, intent in zip(x, y):
        pred_response = bot_my_version.predict(message)
        pred = models.PromptResponse(**pred_response)
        if pred.intent == intent:
            count += 1
    return count / len(x)

Ovdje spajamo train dataset i "teški" test dataset (dataset_test_hard.csv), pa želimo vidjeti hoće li model bolje radit od  77.27 % koliko je imao kad je samo bil treniran na train datasetu i onda testiran na "dataset_test_easy.csv"


In [4]:
x1, y1 = data_handler.get_train_data_xy()
x2, y2 = data_handler.get_test_data_hard_xy()

x = pd.concat([x1, x2], ignore_index=True)
y = pd.concat([y1, y2], ignore_index=True)
bot_my_version._bundle = bot_my_version._build_pipeline(X = x,y = y)


making new pipeline, x: 85 y: 85


/Users/tomislavziger/Developer/rba-chatbot-assignment/.venv/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


In [5]:
x,y = data_handler.get_test_data_easy_xy()
print("Točnost: ", get_accuracy(x,y) * 100 , "%")


Točnost:  86.36363636363636 %


Evo sa 77.27% na 86.36% smo uspjeli točnost povećati samo sa većim datasetom


Zanimljivo bi bilo testirati kakva je točnost na test_hard ako treniramo sa test_easy i train, samo sa train je bila jako loša točnost 22.22%


In [6]:
x1, y1 = data_handler.get_train_data_xy()
x2, y2 = data_handler.get_test_data_easy_xy()

x = pd.concat([x1, x2], ignore_index=True)
y = pd.concat([y1, y2], ignore_index=True)
bot_my_version._bundle = bot_my_version._build_pipeline(X = x,y = y)

x,y = data_handler.get_test_data_hard_xy()
print("Točnost :", get_accuracy(x,y) * 100 , "%")


making new pipeline, x: 44 y: 44
Točnost : 41.269841269841265 %


/Users/tomislavziger/Developer/rba-chatbot-assignment/.venv/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


vidimo da i ovdje veći dataset daje bolje rezultate od 41.27 % u odnosu na prijašnjih 22.22% kad je trenirano samo s originalnim train datasetom



Sad kad smo vidli da veći dataset daje bolje rezultate, idemo spojiti sve u jedan veliki dataset i nasumicno podijelit na train i test, prije nego krenemo dalje

In [7]:
from sklearn.model_selection import train_test_split

x, y = data_handler.get_combined_data_xy()

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, shuffle=True, random_state=42, stratify=y)

print("duljina train skupa", len(X_train))
print("duljina test skupa",len(X_test))




duljina train skupa 85
duljina test skupa 22


i sad vidimo koji je rezultat za nasumičnu podjelu

In [8]:
bot_my_version._bundle = bot_my_version._build_pipeline(X = X_train,y = y_train)
print("Accuracy :", get_accuracy(X_test,y_test) * 100 , "%")

making new pipeline, x: 85 y: 85
Accuracy : 72.72727272727273 %


/Users/tomislavziger/Developer/rba-chatbot-assignment/.venv/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


### Proširivanje skupa podataka

Sada idemo krenuti proširivati naš skup podataka u nadi da će poboljšati naše rezultate

In [9]:
data_augmenter = util.DataAugmenter()

print(type(X_train))

<class 'pandas.core.series.Series'>


In [11]:
X,y = data_augmenter.remove_first(X_train,y_train)

print(X)
print(y)
    

0                                 dje mogu popiti kavu?
1                           mate li popust za studente?
2     mam osobnog asistenta zbog invaliditeta, jel m...
3                         ostoji li popust za studente?
4     mam ozljedu koljena od previše igranja šaha i ...
                            ...                        
80              el ima gdi u muzeju za kupit grickalice
81                       ostoji li popust za branitelje
82    ntschuldigung, Ja biti Stefan iz Minken, imati...
83    olika je cijena članarine, ja sam inače nećak ...
84                           el ima popust za invalide?
Length: 85, dtype: object
0             kafic
1          ulaznice
2     pristupacnost
3          ulaznice
4     pristupacnost
          ...      
80            kafic
81         ulaznice
82          kontakt
83         clanstvo
84         ulaznice
Length: 85, dtype: object
